In [8]:
import asyncio , re , httpx
from bs4 import BeautifulSoup
from typing import Optional , AsyncIterator , List
import urllib.parse
import requests
from dotenv import load_dotenv

In [9]:
env_variables = load_dotenv()

In [ ]:
wiley_key 

In [2]:
import sys, os
sys.path.append(os.path.abspath('..'))
from functions.paper_to_doi import get_info_from_doi


2025-06-14 08:47:52.766480: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749912472.830036 1844935 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749912472.847407 1844935 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749912472.966017 1844935 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749912472.966044 1844935 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749912472.966047 1844935 computation_placer.cc:177] computation placer alr

In [ ]:
# class Easy_Publisher_PDF_Resolver:
#     """
#     Resolve a DOI to a direct PDF URL for Springer, OUP, Hindawi, F1000, etc.
#     Raises NeedsBrowser when the easy path fails.
#     """
    
#     _SPRINGER_HOST     = "link.springer.com"
#     _OUP_HOST_RE       = re.compile(r"^https?://academic\.oup\.com/")
#     _F1000_HOST_RE     = re.compile(r"^https?://(?:f1000research|wellcomeopenresearch|gatesopenresearch)\.org/")
#     _HINDAWI_DOWNLOAD  = re.compile(r"^https?://downloads\.hindawi\.com/")
#     _HINDAWI_LANDING   = re.compile(r"^https?://(?:www\.)?hindawi\.com/")

    
#     _ANCHOR_PDF_CLASS_RE = re.compile(r"pdf|download", re.I)  # class / id hints


#     class NeedsBrowser(RuntimeError):
#         """Raised when the resolver needs a browser to resolve the URL."""
#         pass
    
    
#     def __init__(self,*,
#                  headers: Optional[dict] = None,
#                  timeout: int = 30):
#         self.headers = headers or {
#             "User-Agent": "Mozilla/5.0 (easy-pdf-resolver/0.2)"
#         }
#         self.timeout = timeout
#         self._client : Optional[httpx.AsyncClient] = None
      
#     async def __aenter__(self): 
#         self._client = httpx.AsyncClient(
#             headers=self.headers,
#             follow_redirects=True, 
#             timeout=self.timeout
#         )
        
#         return self
    
#     async def __aexit__(self, exc_type, exc, traceback):
#         await self._client.aclose()
#         self._client = None
        
#     def _require_client(self) -> httpx.AsyncClient:
#         if self._client is None:
#             # allow use without 'async with' by lazily creating a client
#             self._client = httpx.AsyncClient(
#                 headers=self.headers,
#                 follow_redirects=True,
#                 timeout=self.timeout
#             )
#         return self._client

    
#     async def _is_pdf(self, url: str) -> bool:
#         client = self._require_client()
#         r = await client.head(url)
#         return (
#             r.status_code == 200 and
#             r.headers.get("content-type", "").startswith("application/pdf")
#         )
    
#     @staticmethod
#     def _extract_meta_pdf(html: str) -> Optional[str]:
#         """
#         Extract the PDF URL advertised in a page’s ``<meta>`` tag.

#         Many scholarly-publisher landing pages include
#         ``<meta name="citation_pdf_url" content="https://…/article.pdf">``  
#         inside their ``<head>`` section.  This helper pulls that value out
#         so the scraper can download the PDF without further heuristics.

#         Args:
#         html (str):
#             Raw HTML source of the article landing page
#             (typically obtained via ``await http_client.get(url).text``).

#         Returns:
#             Optional[str]:
#                 • The PDF URL found in the ``content`` attribute of the first
#                 ``<meta name="citation_pdf_url">`` tag, **or**  
#                 • ``None`` if no such tag exists (caller should try another
#                 strategy, e.g. Playwright).

#         Example:
#             >>> landing_html = client.get("https://academic.oup.com/jn/article/...", timeout=30).text
#             >>> pdf_url = _extract_meta_pdf(landing_html)
#             >>> print(pdf_url)
#             https://academic.oup.com/jn/article-pdf/123/4/1234/9876543/jn.123.4.1234.pdf

#         Notes:
#             * Uses BeautifulSoup’s default parser (``html.parser``) to avoid
#             external dependencies.  Switch to ``"lxml"`` if you already have
#             lxml installed and want a performance boost.
#             * Only the **first** matching tag is returned; this is sufficient
#             for OUP, Wiley HTML, and most CrossRef-compliant publishers.

#         """
#         soup = BeautifulSoup(html, "html.parser")
#         tag  = soup.find("meta", attrs={"name": "citation_pdf_url"})
#         return tag["content"] if tag else None
    
    
#     @classmethod
#     def _extract_anchor_pdf(cls, html: str, base_url: str) -> Optional[str]:
#         """Return absolute URL from the *best* `<a>` that seems to be a PDF link.

#         Heuristics (checked in order):
#         1. `href` ends with `.pdf` (common for Springer / OUP).
#         2. `href` contains `/pdf` *and* anchor text or class‑attribute mentions
#            "pdf" or "download" (F1000 style: `/pdf?article_uuid=…`).
#         3. Anchor `class` / `id` matches /pdf|download/ even if `href` is just
#            a relative `/content/pdf/…` path.
#         """
#         soup = BeautifulSoup(html, "html.parser")

#         def make_abs(href: str) -> str:
#             return urllib.parse.urljoin(base_url, href)

#         # Pass 0 – explicit OUP pattern /article-pdf/ ---------------------------
#         for a in soup.find_all("a", href=True):
#             href = a["href"].strip()
#             if "/article-pdf/" in href.lower():
#                 return make_abs(href)

#         # Pass 1 – href ends with .pdf -----------------------------------------
#         for a in soup.find_all("a", href=True):
#             href = a["href"].strip()
#             if href.lower().endswith(".pdf"):
#                 return make_abs(href)

#         # Pass 2 – href contains "pdf" + textual or class cues -----------------
#         for a in soup.find_all("a", href=True):
#             href = a["href"].strip()
#             href_lower = href.lower()
#             if "pdf" in href_lower:
#                 text = a.get_text(" ", strip=True).lower()
#                 classes = " ".join(a.get("class", [])).lower()
#                 ident   = a.get("id", "").lower()
#                 if ("pdf" in text or "download" in text or cls._ANCHOR_HINT_RE.search(classes) or cls._ANCHOR_HINT_RE.search(ident)):
#                     return make_abs(href)

#         # Pass 3 – only class/id hints -----------------------------------------
#         for a in soup.find_all("a", href=True):
#             classes = " ".join(a.get("class", []))
#             ident   = a.get("id", "")
#             if cls._ANCHOR_HINT_RE.search(classes) or cls._ANCHOR_HINT_RE.search(ident):
#                 return make_abs(a["href"].strip())

#         return None

        
       
        
#     async def get_pdf(self, doi: str) -> str:
        
#         """
#         Return a direct PDF URL for the given DOI.
#         Raises EasyPDFResolver.NeedsBrowser if the easy strategies fail.
#         """
#         client = self._client or httpx.AsyncClient(headers=self.headers, follow_redirects=True, timeout=self.timeout)
#         landing = str((await client.get(f"https://doi.org/{doi}")).url)
        
#         if self._SPRINGER_HOST in landing:
#             candidates = [
#                 f"https://{self._SPRINGER_HOST}/content/pdf/{doi}.pdf",
#             ]
#             # /fulltext.html → .pdf
#             if landing.endswith("fulltext.html") or landing.endswith("fulltext.htm"):
#                 candidates.append(landing.rsplit("/", 1)[0] + ".pdf")
#             # article path without extension → add .pdf
#             elif landing.endswith(".html"):
#                 candidates.append(landing[:-5] + ".pdf")
#             else:
#                 if "/article/" in landing and not landing.endswith(".pdf"):
#                     candidates.append(landing + ".pdf")

#             for pdf_url in candidates:
#                 if await self._is_pdf(pdf_url):
#                     return pdf_url

#             # 🚀 NEW: scrape the Download‑PDF button

#         # ------------------------------------------------------------------
#         # Hindawi – DOI redirect or downloads.hindawi.com
#         # ------------------------------------------------------------------
#         if self._HINDAWI_DOWNLOAD.match(landing) or self._HINDAWI_LANDING.match(landing):
#             for cand in {landing, f"https://doi.org/{doi}"}:
#                 if await self._is_pdf(cand):
#                     return cand
#             # try landing→html parse as last resort
#             html = (await client.get(landing)).text
#             meta = self._extract_meta_pdf(html)
#             if meta and await self._is_pdf(meta):
#                 return meta

#         # ------------------------------------------------------------------
#         # F1000Research family – JSON API
#         # ------------------------------------------------------------------
#         if self._F1000_HOST_RE.match(landing):
#             article_id = doi.split("/")[-1]
#             host = urllib.parse.urlparse(landing).hostname
#             api_url = f"https://api.{host}/article/{article_id}"
#             try:
#                 pdf_url = (await client.get(api_url)).json()["data"]["pdf_url"]
#                 if await self._is_pdf(pdf_url):
#                     return pdf_url
#             except Exception:
#                 pass

#         # ------------------------------------------------------------------
#         # OUP – meta tag only
#         # ------------------------------------------------------------------
#         if self._OUP_HOST_RE.match(landing):
#             html = (await client.get(landing)).text
#             meta_link = self._extract_meta_pdf(html)
#             if meta_link and await self._is_pdf(meta_link):
#                 return meta_link
#         try:
#             html = (await client.get(landing)).text
#             anchor_link = self._extract_anchor_pdf(html, landing)
#             if anchor_link and await self._is_pdf(anchor_link):
#                 return anchor_link
#         except httpx.HTTPError:
#             pass 
#         # ------------------------------------------------------------------
#         # Escalate
#         # ------------------------------------------------------------------
#         raise self.NeedsBrowser(f"Browser required for {doi} → {landing}")


In [ ]:
class EasyPDFResolver:
    _SPRINGER_HOST = "link.springer.com"
    _OUP_HOST_RE = re.compile(r"^https?://academic\.oup\.com/")
    _F1000_HOST_RE = re.compile(r"^https?://(?:f1000research|wellcomeopenresearch|gatesopenresearch)\.org/")
    _HINDAWI_DOWNLOAD_RE = re.compile(r"^https?://downloads\.hindawi\.com/")
    _HINDAWI_LANDING_RE = re.compile(r"^https?://(?:www\.)?hindawi\.com/")
    _ANCHOR_HINT_RE = re.compile(r"pdf|download", re.I)
    wiley_token = env_variables.get("wiley_api_token")

    class NeedsBrowser(RuntimeError):
        pass

    def __init__(self, *, headers: Optional[dict] = None, timeout: int = 30):
        self.headers = headers or {"User-Agent": "Mozilla/5.0 (easy-pdf-resolver/1.2)"}
        self.timeout = timeout
        self._client: Optional[httpx.AsyncClient] = None

    async def __aenter__(self):
        self._client = httpx.AsyncClient(headers=self.headers, follow_redirects=True, timeout=self.timeout)
        return self

    async def __aexit__(self, exc_type, exc, tb):
        if self._client:
            await self._client.aclose()
            self._client = None

    def _client_required(self) -> httpx.AsyncClient:
        if self._client is None:
            self._client = httpx.AsyncClient(headers=self.headers, follow_redirects=True, timeout=self.timeout)
        return self._client

    async def get_pdf(self, doi: str) -> str:
        client = self._client_required()
        landing = str((await client.get(f"https://doi.org/{doi}")).url)

        if self._SPRINGER_HOST in landing:
            for url in self._springer_candidates(landing, doi):
                if await self._is_pdf(url):
                    return url

        if self._HINDAWI_DOWNLOAD_RE.match(landing) or self._HINDAWI_LANDING_RE.match(landing):
            for url in (landing, f"https://doi.org/{doi}"):
                if await self._is_pdf(url):
                    return url

        if self._F1000_HOST_RE.match(landing):
            api_pdf = await self._f1000_pdf(landing, doi)
            if api_pdf and await self._is_pdf(api_pdf):
                return api_pdf

        if self._OUP_HOST_RE.match(landing):
            oup_pdf = await self._oup_pdf(landing)
            if oup_pdf:
                return oup_pdf

        html = (await client.get(landing)).text
        anchor = self._extract_anchor_pdf(html, landing)
        if anchor:
            return anchor

        cr_pdf = self._crossref_fallback(doi)
        if cr_pdf:
            return cr_pdf

        raise self.NeedsBrowser(f"Browser required for {doi} → {landing}")

    def _springer_candidates(self, landing: str, doi: str) -> List[str]:
        base = f"https://{self._SPRINGER_HOST}"
        candidates = [f"{base}/content/pdf/{doi}.pdf"]
        if landing.endswith(("fulltext.html", "fulltext.htm")):
            candidates.append(landing.rsplit("/", 1)[0] + ".pdf")
        elif landing.endswith(".html"):
            candidates.append(landing[:-5] + ".pdf")
        elif "/article/" in landing and not landing.endswith(".pdf"):
            candidates.append(landing + ".pdf")
        return candidates

    async def _f1000_pdf(self, landing: str, doi: str) -> Optional[str]:
        client = self._client_required()
        article_id = doi.split("/")[-1]
        host = urllib.parse.urlparse(landing).hostname
        api_url = f"https://api.{host}/article/{article_id}"
        try:
            r = await client.get(api_url)
            r.raise_for_status()
            return r.json()["data"]["pdf_url"]
        except Exception:
            return None

    async def _oup_pdf(self, landing: str) -> Optional[str]:
        client = self._client_required()
        html = (await client.get(landing)).text
        meta = self._extract_meta_pdf(html)
        if meta:
            return meta
        anchor = self._extract_anchor_pdf(html, landing)
        return anchor

    async def _wiley_pdf(self , landing:str , doi:str) -> Optional[str]:
        client = self._client_required()
        
        #1 TDM Api
        api_url = f"https://api.wiley.com/tdm/v1/articles?{doi}/pdf"
        r = await client.get(
            api_url,
            headers= {"Authorization": f"Bearer {self.wiley_token}"})
        if r.status_code == 200 and r.headers.get("content-type", "").startswith("application/pdf"):
            return api_url
        
        #pdfdirect 
        
        pdf_url = f"https://onlinelibrary.wiley.com/doi/pdfdirect/{doi}"
        r2 = await client.get(pdf_url)
        if r2.status_code == 200 and r2.headers.get("content-type", "").startswith("application/pdf"):
            return pdf_url
        
        #htmlparse 
        
        html = (await client.get(landing)).text
        soup = BeautifulSoup(html, "html.parser")
        btn = soup.find("a", class_="pdf-download-link", href=True)
        if btn:
            return urllib.parse.urljoin(landing, btn["href"])
        return None 
    
    async def _tand_pdf(self , landing:str , doi:str) -> Optional[str]:
        client = self._client_required()
        
        #direct pdf endpoint 
        
        pdf_endpoint = f"https://www.tandfonline.com/doi/pdf/{doi}"
        resp = await client.get(pdf_endpoint)
        if resp.status_code == 200 and resp.headers.get("content-type", "").startswith("application/pdf"):
            return pdf_endpoint
        
        #meta tag
        html = (await client.get(landing)).text
        if (meta:= self._extract_meta_pdf(html)):
            return urllib.parse.urljoin(landing, meta)
        
        if (anchor := self._extract_anchor_pdf(html, landing)):
            return anchor
        return None
        
    async def _sage_pdf(self, landing: str, doi: str) -> Optional[str]:
        client = self._client_required()
        
        #pdf endpoint
        
        pdf_url = f"https://journals.sagepub.com/doi/pdf/{doi}"
        r = await client.get(pdf_url)
        if r.status_code == 200 and r.headers.get("content-type", "").startswith("application/pdf"):
            return pdf_url
        
        html = (await client.get(landing)).text
        if (meta:= self._extract_meta_pdf(html)):
            return urllib.parse.urljoin(landing, meta)
        
        if (anchor := self._extract_anchor_pdf(html, landing)):
            return anchor
        return None
        
        
    async def _karger_pdf(self, landing: str, doi: str) -> Optional[str]:
        clinet = self._client_required()
        
        #must get article id 
        
        html = (await clinet.get(landing)).text
        soup = BeautifulSoup(html, "html.parser")
        tag = soup.find("meta", attrs={"name": "dc.identifier"})
        if not tag or not tag.get("data-article-id"):
            return None
        article_id = tag["data-article-id"]
        return f"https://www.karger.com/Article/Pdf/{article_id}"
       
    async def _is_pdf(self, url: str) -> bool:
        client = self._client_required()
        try:
            r = await client.head(url)
            if r.status_code in (403, 405):
                r = await client.get(url, headers={"Range": "bytes=0-0"})
            return r.status_code == 200 and r.headers.get("content-type", "").startswith("application/pdf")
        except Exception:
            return False

    @staticmethod
    def _extract_meta_pdf(html: str) -> Optional[str]:
        tag = BeautifulSoup(html, "html.parser").find("meta", attrs={"name": "citation_pdf_url"})
        return tag["content"].strip() if tag and tag.get("content") else None

    def _extract_anchor_pdf(self, html: str, base_url: str) -> Optional[str]:
        soup = BeautifulSoup(html, "html.parser")
        join = lambda h: urllib.parse.urljoin(base_url, h)
        for a in soup.find_all("a", href=True):
            href = a["href"].strip()
            lower = href.lower()
            full_url = join(href)
            if lower.endswith(".pdf") or "/article-pdf/" in lower or "/advance-article-pdf/" in lower:
                return full_url
            text = a.get_text(" ").lower()
            cls = " ".join(a.get("class", [])).lower()
            ident = a.get("id", "").lower()
            if "download pdf" in text or self._ANCHOR_HINT_RE.search(cls) or self._ANCHOR_HINT_RE.search(ident):
                return full_url
        return None

    @staticmethod
    def _crossref_fallback(doi: str) -> Optional[str]:
        try:
            r = requests.get(f"https://api.crossref.org/works/{doi}", timeout=15)
            r.raise_for_status()
            data = r.json()
        except Exception:
            return None
        links = data.get("message", {}).get("link", [])
        best = None
        for link in links:
            ct = link.get("content-type")
            url = link.get("URL")
            if ct == "text/html":
                return url
            if ct == "application/pdf" and best is None:
                best = url
            if url and url.endswith(".pdf") and best is None:
                best = url
        return best

In [4]:
test_dois = [
    "10.1155/2022/2836128" # Hindawi OA
]

In [5]:
from pprint import pprint
async with EasyPDFResolver() as r:
    pdf_url = await r.get_pdf(test_dois[0])
    pprint(pdf_url)

'http://downloads.hindawi.com/journals/jir/2022/2836128.pdf'


In [6]:
paper_info = get_info_from_doi("10.1093/rheumatology/keab153")

In [7]:
paper_info

{'doi': '10.1093/rheumatology/keab153',
 'title': 'Rituximab plus leflunomide in rheumatoid arthritis: a randomized, placebo-controlled, investigator-initiated clinical trial (AMARA study)',
 'document_link': 'http://academic.oup.com/rheumatology/advance-article-pdf/doi/10.1093/rheumatology/keab153/36643438/keab153.pdf'}